In [5]:
from torch.utils.data import DataLoader, random_split,TensorDataset
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import NN_classes
from torchvision import datasets, transforms
import training_methods
import DataLoading



In [10]:
#executions = ["Network_Line_Out_N_101_N_102_cac1","Network_Line_Out_N_102_N_104_cac1","Network_Line_Out_N_101_N_105_cac1","Network_Line_Out_N_102_N_104_cac1","Network_Line_Out_N_102_N_106_cac1","Network_Line_Out_N_103_N_109_cac1"]
#executions = ["Network_Line_Out_N_101_N_102_cac1"]
#executions = ["Network_Full_Generation_Full","Network_Line_In_N_101_N_102_cac1","Network_Line_In_N_101_N_103_cac1","Network_Line_In_N_101_N_105_cac1"]

executions = ["Network_Existing_Generation_Full"]
sc = "sc01"
period = "2030"
folder = "Samples_3-bus_ACOPF"
te_s = 0.1
val_s = 0.2

In [11]:
dfs_in,dfs_out = DataLoading.load_data(folder,executions,period,sc)

Data/input_f_sc01_Network_Existing_Generation_Full_2030.csv
23


In [12]:
ts_in,ts_out =  DataLoading.split_tr_val_te(dfs_in,dfs_out,executions,te_s,val_s)

In [13]:
d_ft_in, d_ft_out = DataLoading.concat_and_normalize(ts_in,ts_out,executions)

In [14]:
train = TensorDataset(d_ft_in['train'].float(), d_ft_out['train'].float())
validation = TensorDataset(d_ft_in['val'].float(), d_ft_out['val'].float())

training_loader = DataLoader(train,batch_size=32)
validation_loader = DataLoader(train,batch_size=32)

In [15]:
d_ft_in['train'].shape

torch.Size([6289, 23])

In [16]:
learning_rates = [0.01*4**i for i in range(4)]
loss_fn = torch.nn.MSELoss()
nbs_e = [4,8]
i=0
nbs_hidden = [0,3]
results = pd.DataFrame()
for nb_e in nbs_e:
    for lr in learning_rates:
        for nb_hidden in nbs_hidden: 
            if nb_hidden == 0: 
                m = NN_classes.ObjectiveEstimator_ANN_Single_layer(input_size=d_ft_in['train'].shape[1],output_size=1)
                
            elif nb_hidden == 3: 
                m = NN_classes.ObjectiveEstimator_ANN_3hidden_layer(input_size=d_ft_in['train'].shape[1],hidden_size1=int(d_ft_in['train'].shape[1]/4),hidden_size2=int(d_ft_in['train'].shape[1]/16),hidden_size3=int(d_ft_in['train'].shape[1]/64),output_size=1)
            m_name = f"OE_{nb_hidden}h_{nb_e}e_{lr}lr"
            optimizer = torch.optim.Adam(m.parameters(), lr=lr)
            train_loss = training_methods.train_multiple_epochs(nb_e,m,training_loader,validation_loader,loss_fn,optimizer,m_name,True)
            
            test_predictions = m(d_ft_in["test"].float())
            test_loss = loss_fn(test_predictions,d_ft_out["test"])
            train_predictions = m(d_ft_in["train"].float())
            train_loss = loss_fn(train_predictions,d_ft_out["train"])
            
            r = pd.DataFrame({"Model_type": m_name,"Epochs": nb_e,"Lr":lr, "Tr_l":train_loss.item(),"Te_l":test_loss.item()},index = [i]
            )
            i+=1
            results = pd.concat([results,r])

EPOCH 1:
  batch 1 loss: 0.0021936340630054475
  batch 101 loss: 0.008084873525513103


C:\Workdir\Programs\Miniconda\envs\jr23\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Workdir\Programs\Miniconda\envs\jr23\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([17])) that is different to the input size (torch.Size([17, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


LOSS train 0.005398804827254843 valid 0.00046850694343447685
EPOCH 2:
  batch 1 loss: 1.848475367296487e-06
  batch 101 loss: 0.000179266222962724
LOSS train 0.00014913108567169114 valid 0.0004656228993553668
EPOCH 3:
  batch 1 loss: 6.460045697167516e-06
  batch 101 loss: 8.090905772746737e-05
LOSS train 7.070910374244851e-05 valid 0.00017688451043795794
EPOCH 4:
  batch 1 loss: 2.352863666601479e-06
  batch 101 loss: 5.0711962428522385e-05
LOSS train 4.5106938424041015e-05 valid 8.6059226305224e-05


C:\Workdir\Programs\Miniconda\envs\jr23\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([874])) that is different to the input size (torch.Size([874, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Workdir\Programs\Miniconda\envs\jr23\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([6289])) that is different to the input size (torch.Size([6289, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Workdir\Programs\Miniconda\envs\jr23\lib\site-packages\torch\nn\init.py:405: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


EPOCH 1:
  batch 1 loss: 7.598357001370459e-09
  batch 101 loss: 3.5267525583027036e-06
LOSS train 1.8042565276598634e-06 valid 1.8270878854309558e-07
EPOCH 2:
  batch 1 loss: 1.7451772293952671e-09
  batch 101 loss: 1.1701225131055537e-07
LOSS train 7.893608026330289e-08 valid 1.0667744732018036e-07
EPOCH 3:
  batch 1 loss: 9.498042174982402e-10
  batch 101 loss: 9.928076639464934e-08
LOSS train 6.448141113781532e-08 valid 4.4329382831165276e-08
EPOCH 4:
  batch 1 loss: 2.7932307133937684e-10
  batch 101 loss: 1.5157064648718687e-07
LOSS train 1.2346435350678596e-07 valid 5.392543656057569e-08
EPOCH 1:
  batch 1 loss: 3.974652849137783e-05
  batch 101 loss: 0.028787242122270983
LOSS train 0.014833312932474137 valid 0.00039810611633583903
EPOCH 2:
  batch 1 loss: 4.707629559561611e-06
  batch 101 loss: 0.0002526248555705024
LOSS train 0.00014245426134355254 valid 2.056378980341833e-05
EPOCH 3:
  batch 1 loss: 6.815429514972493e-08
  batch 101 loss: 2.508340661620423e-05
LOSS train 4.46

LOSS train 1.2409942332979863e-07 valid 5.547791204207897e-08
EPOCH 5:
  batch 1 loss: 4.0238514742441114e-10
  batch 101 loss: 7.136823384268353e-08
LOSS train 7.308476398013823e-08 valid 3.924785474396231e-08
EPOCH 6:
  batch 1 loss: 2.2196834592591587e-10
  batch 101 loss: 1.1251271335499524e-07
LOSS train 1.202225075047636e-07 valid 4.7909562539416584e-08
EPOCH 7:
  batch 1 loss: 2.288227740621096e-10
  batch 101 loss: 2.0354335726890939e-07
LOSS train 1.4458170020492937e-07 valid 5.576764294801251e-08
EPOCH 8:
  batch 1 loss: 2.9960183667299133e-10
  batch 101 loss: 1.3484949927144995e-07
LOSS train 1.497750838321911e-07 valid 1.3454170755267114e-07
EPOCH 1:
  batch 1 loss: 0.012243911027908325
  batch 101 loss: 0.20678476940327528
LOSS train 0.111556607448615 valid 0.0005304967053234577
EPOCH 2:
  batch 1 loss: 2.073441428365186e-06
  batch 101 loss: 0.0001968661233331659
LOSS train 0.00011516872118671775 valid 1.856757990026381e-05
EPOCH 3:
  batch 1 loss: 1.3906805179431103e-07

In [11]:
results

,Model_type,Epochs,Lr,Tr_l,Te_l
0,sl,4,0.01,1.995044e-02,3.113820e-02
1,3h,4,0.01,6.880393e-08,5.301107e-08
2,sl,4,0.04,2.364681e-04,2.971305e-04
3,3h,4,0.04,6.932472e-08,5.343779e-08
4,sl,4,0.16,2.749902e-05,2.335273e-05
5,3h,4,0.16,6.936998e-08,5.347490e-08
6,sl,4,0.64,2.291427e-04,1.987444e-04
7,3h,4,0.64,6.930760e-08,5.342376e-08
8,sl,8,0.01,2.233874e-06,1.735164e-06
9,3h,8,0.01,2.290619e-08,3.574458e-08
